# Librerias utilizadas

In [ ]:
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:xplot.plotly.interactive"
#r "nuget: HtmlAgilityPack, 1.11.42"
#r "nuget: ScrapySharp, 3.0.0"

using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using HtmlAgilityPack;
using ScrapySharp.Extensions;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using XPlot.Plotly;


Installed Packages HtmlAgilityPack, 1.11.42 Microsoft.Data.Analysis, 0.4.0 ScrapySharp, 3.0.0 xplot.plotly.interactive, 4.0.6

## Para obtener los lenguajes que cumplan con la condicion topic:"TOPIC"

In [ ]:
//  Dada una cadena y un patron a buscar retorna una lista en forma de tupla donde se encuentra los indices
// (inicio, fin) de la aparicion de ese patron en la cadena str
static List<Tuple<int,int>> AllIndexesOf(string str, string value)
        {
            if (String.IsNullOrEmpty(value))
                throw new ArgumentException("La cadena no puede ser vacia", "value");
            List<Tuple<int, int>> indexes = new List<Tuple<int, int>>();
            for (int index = 0; ; index += value.Length)
            {
                index = str.IndexOf(value, index);
                var index2 = str.IndexOf('"',index+7);
                if (index == -1)
                    return indexes;
                indexes.Add(new Tuple<int, int>(index, index2));
            }
        }

In [ ]:
            string fileName = @"./topics.txt";
            FileInfo fi = new FileInfo(fileName);

            string fileName2 = @"./plotTema2.csv";
            FileInfo fi2 = new FileInfo(fileName2);
            Dictionary<string, int> topics = new Dictionary<string, int>();

            var url = "https://github.com/topics/data-structures?o=desc&s=updated";
            //iniciar el recorrido por pagina desde la 1 hasta la 20
            for (int i = 1; i < 11; i++)
            {
                HtmlWeb web = new HtmlWeb();
                HtmlDocument html = web.Load(url+"&page="+i.ToString());

                //obtener los css que contienen los topics y la fecha de publicacion
                var nodes = html.DocumentNode.CssSelect("[class='color-bg-default rounded-bottom-2']").Select(x => x.InnerHtml).Distinct();

                var nodos = nodes.ToList();

                //recorrer los nodos para obtener el datetipe y el conjunto de topicos
                foreach (var nodo in nodos)
                {
                    int firstStringPosition = nodo.IndexOf("datetime");
                    string iDate = nodo.Substring(firstStringPosition + 10,
                        10);
                    var updated = DateTime.Parse(iDate);
                    if ((DateTime.Now - updated).TotalDays < 30)
                    {
                        //Console.WriteLine(updated);
                        //indices de los topicos
                        var indices = AllIndexesOf(nodo, "Topic:");

                        foreach (Tuple<int, int> tuple in indices)
                        {
                            string topic = nodo.Substring(tuple.Item1 + 7, tuple.Item2- tuple.Item1-7);
                            //Console.WriteLine(topic);
                            if (topics.ContainsKey(topic))
                            {
                                topics[topic]++;
                            }
                            else
                            {
                                topics.Add(topic, 1);
                            }
                        }

                    }
                }
            }
            //ordenar la lista
            var ordered = topics.OrderBy(x => x.Value).ToDictionary(x => x.Key, x => x.Value);
            var descendente = ordered.Reverse();

            foreach (var d in descendente)
            {
                Console.WriteLine($"{d.Key},{d.Value}");
            }

            try
            {
                // Si el archivo ya existe eliminar
                if (fi.Exists)
                {
                    fi.Delete();
                }

                // Crear el archivo para guardar los resultados    
                using (StreamWriter sw = fi.CreateText())
                {
                    foreach (var d in descendente)
                    {
                        sw.WriteLine(d.Key);
                    }
                }
            }
            catch (Exception Ex)
            {
                Console.WriteLine(Ex.ToString());
            }

            try
            {
                // Si el archivo ya existe eliminar
                if (fi2.Exists)
                {
                    fi2.Delete();
                }

                // Crear para el segundo archivo 
                using (StreamWriter sw2 = fi2.CreateText())
                {
                    sw2.WriteLine("Topic,Apariciones");
                    var cont = 0;
                    foreach (var d in descendente)
                    {
                        sw2.WriteLine(d.Key+","+d.Value);
                        cont++;
                        if(cont == 20)
                        {
                            break;
                        }
                    }
                }
            }
            catch (Exception Ex)
            {
                Console.WriteLine(Ex.ToString());
            }

data-structures,295
algorithms,178
leetcode,72
python,53
algorithm,51
java,51
cpp,50
javascript,38
leetcode-solutions,33
problem-solving,29
competitive-programming,25
computer-science,24
c,17
algorithms-and-data-structures,17
dsa,15
python3,15
stack,14
linked-list,13
interview-questions,12
interview,11
queue,11
agpl,11
hackerrank,9
sql,9
rust,8
interview-preparation,8
codeforces,8
database,8
typescript,8
programming,8
golang,8
leetcode-cpp,8
algorithm-challenges,7
graph,7
heap,7
hacktoberfest,6
practice,6
leetcode-java,6
dsa-algorithm,6
dsa-practice,6
dynamic-programming,6
data-structure,6
sorting-algorithms,6
array,6
data-structures-and-algorithms,6
geeksforgeeks,5
binary-search-tree,5
oop,5
binary-trees,5
tree,5
heaps,5
hackerrank-solutions,4
datastructures,4
kotlin,4
algorithms-datastructures,4
codewars,4
solutions,4
design-patterns,4
data-science,4
go,4
sort,4
nodejs,4
red-black-tree,4
searching-algorithms,4
leetcode-questions,4
coding,4
priority-queue,4
online-judge,3
object-orien

# Grafico Topic Vs. Apariciones

In [ ]:
var df1 = DataFrame.LoadCsv("./plotTema2.csv");
var chart = XPlot.Plotly.Chart.Plot(
    new Bar()
    {
        x = df1.Columns["Topic"],
        y = df1.Columns["Apariciones"],
        marker = new XPlot.Plotly.Marker()
        {
            color = "green",
            colorscale = "Jet"
        }
    }
);

chart.Width = 500;
chart.Height = 600;
chart.WithXTitle("Topic");
chart.WithYTitle("Apariciones");
chart.WithTitle("Topic vs. Apariciones");
chart.Display();

<!DOCTYPE html>